In [9]:
!pip install boto3 --quiet

In [10]:
from lib import create_tweet_iterator

In [4]:
CONSUMER_KEY = "jNQiuHX128wrIEwGNRefUNdCu"
CONSUMER_SECRET = "jZtWy8x4hU5rfhTPXXFuTEMuKrc6cgzi2mLEYkthPgTYHDiTrr"
ACCESS_TOKEN = "1019793622953222145-AKiXmsPpBVajjRThKEafxqi8Kv9Zvd"
ACCESS_SECRET = "PWXH26H61SM44ggxJhYVwlndYP9Qnw6xsv9c3nWdz2ZnS"

In [11]:
los_angeles_bbox = "-118.551346,33.96666,-118.443428,34.05056"

In [12]:
from lib import create_tweet_iterator

tweet_iterator = create_tweet_iterator(ACCESS_TOKEN, ACCESS_SECRET, CONSUMER_KEY, CONSUMER_SECRET, los_angeles_bbox)

In [13]:
tweet = next(tweet_iterator)

In [16]:
tweet.keys()

dict_keys(['created_at', 'id', 'id_str', 'text', 'display_text_range', 'source', 'truncated', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'quoted_status_id', 'quoted_status_id_str', 'quoted_status', 'quoted_status_permalink', 'is_quote_status', 'quote_count', 'reply_count', 'retweet_count', 'favorite_count', 'entities', 'favorited', 'retweeted', 'possibly_sensitive', 'filter_level', 'lang', 'timestamp_ms'])

In [20]:
tweet['text']

'@BlackZack93 🤷🏽\u200d♀️😂 https://t.co/luk1hFYgtr'

In [25]:
tweets = []
for i in range(20):
    tweet = next(tweet_iterator)
    tweets.append(tweet)

In [26]:
import json
from lib import create_timestamped_filename

In [27]:
username = "sangyun_jacob"
filename = create_timestamped_filename(username)

with open(filename, 'w') as outfile:
    json.dump(tweets, outfile)

In [32]:
filename

'tweets-sangyun_jacob-2018-07-24_02-35-06-149229.json'

In [28]:
from lib import create_boto_client

In [29]:
s3_client = create_boto_client()

In [31]:
from lib import write_file_to_S3

In [33]:
filename = 'tweets-sangyun_jacob-2018-07-24_02-35-06-149229.json'
S3_BUCKET = 'uclax-data-science'

In [34]:
write_file_to_S3(s3_client, filename, S3_BUCKET)

In [35]:
from lib import list_files_in_S3_bucket
S3_BUCKET = 'uclax-data-science'

list_files_in_S3_bucket(s3_client, S3_BUCKET)

['data.json',
 'tweets-ChillSimmons-2018-07-24_02-33-28-092154.json',
 'tweets-albert-2018-07-24_02-21-19-501738.json',
 'tweets-dennissibo-2018-07-24_02-12-06-781711.json',
 'tweets-jia-2018-07-24_02-20-18-817448.json',
 'tweets-joshua-2018-07-21_14-09-23-363540.json',
 'tweets-joshua-2018-07-23_11-22-20-369005.json',
 'tweets-joshua-2018-07-23_11-59-41-971097.json',
 'tweets-joshua-2018-07-24_02-21-29-011479.json',
 'tweets-kristine-2018-07-24_02-21-53-840324.json',
 'tweets-liling-2018-07-24_02-31-04-417107.json',
 'tweets-mark-2018-07-24_02-37-46-350548.json',
 'tweets-mryan-2018-07-24_02-14-02-454519.json',
 'tweets-nina-2018-07-24_02-16-10-999671.json',
 'tweets-nina-2018-07-24_02-36-32-438954.json',
 'tweets-sangyun_jacob-2018-07-24_02-35-06-149229.json',
 'tweets-stanley-2018-07-24_02-04-13-008476.json',
 'tweets-stanley-2018-07-24_02-22-16-555638.json',
 'tweets-stanley-2018-07-24_02-22-54-081139.json',
 'tweets-stanley-2018-07-24_02-25-59-012050.json',
 'tweets-yan-2018-07-24

In [36]:
from lib import read_object_from_S3

In [54]:
S3_BUCKET = 'uclax-data-science'

In [55]:
key = filename

In [56]:
tweets_from_s3 = read_object_from_S3(s3_client, key, S3_BUCKET)

In [58]:
from lib import create_mongo_client_to_database_collection

In [60]:
len(tweets_from_s3)

20

In [59]:
collection_client = create_mongo_client_to_database_collection('twitter','tweets')
collection_client.insert_one(tweet)

In [72]:
tweet=[]
length = len(tweets_from_s3)

for i in range(length):
    tweet = tweets_from_s3[i]
    collection_client.insert_one(tweet)

In [74]:
from pymongo.errors import DuplicateKeyError

In [77]:
tweets = []
try:
    for tweet in tweets_from_s3:
        collection_client.insert_one(tweet)
except DuplicateKeyError:
    pass